<a href="https://colab.research.google.com/github/DaheePark0415/Econ512-Fall2024/blob/main/IO_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probome 1(a). Multinomial Logit

In [29]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the data
df = pd.read_csv('product_data.csv')

# Check the structure of the data to ensure it has the correct columns
print(df.head())

   product_ID  nest     price     sugar  caffeine  market_share  \
0           1  Diet  2.814362  0.631224  6.752525      0.111141   
1           2  Diet  2.935735  0.004553  6.784396      0.081787   
2           3  Diet  2.467309  0.739947  5.761261      0.085727   
3           4  Diet  1.543958  0.103660  4.468299      0.007187   
4           5  Diet  1.495961  0.971926  4.052750      0.015912   

   caffeine_extract_price  corn_syrup_price  t  
0                0.267468          0.251714  1  
1                0.320000          0.253146  1  
2                0.252531          0.314781  1  
3                0.203220          0.227481  1  
4                0.156466          0.244453  1  


In [30]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize


In [31]:
# Load the dataset
df = pd.read_csv('/content/product_data.csv')

# Show the first few rows of the dataset to verify
print(df.head())


   product_ID  nest     price     sugar  caffeine  market_share  \
0           1  Diet  2.814362  0.631224  6.752525      0.111141   
1           2  Diet  2.935735  0.004553  6.784396      0.081787   
2           3  Diet  2.467309  0.739947  5.761261      0.085727   
3           4  Diet  1.543958  0.103660  4.468299      0.007187   
4           5  Diet  1.495961  0.971926  4.052750      0.015912   

   caffeine_extract_price  corn_syrup_price  t  
0                0.267468          0.251714  1  
1                0.320000          0.253146  1  
2                0.252531          0.314781  1  
3                0.203220          0.227481  1  
4                0.156466          0.244453  1  


In [35]:
# Define the utility function with corrected handling for Diet and Regular using the 'nest' column
def utility(params, price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)  # Assuming the other category is Regular
    return alpha * price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Define the log-likelihood function for the multinomial logit model
def log_likelihood(params, df):
    # Calculate utility for each product in each time period
    df['utility'] = utility(params, df['price'], df['sugar'], df['caffeine'], df['nest'])

    # Exponentiate utility
    df['exp_utility'] = np.exp(df['utility'])

    # Group by time period, sum over products to get the denominator for each period
    df['sum_exp_util'] = df.groupby('t')['exp_utility'].transform('sum') + 1  # Add 1 for the outside option

    # Probability of choosing product j at time t
    df['prob'] = df['exp_utility'] / df['sum_exp_util']

    # Log of the probability weighted by quantity (here using 'market_share')
    df['log_prob'] = np.log(df['prob']) * df['market_share']

    # Negative log-likelihood (since we minimize)
    return -df['log_prob'].sum()

# Initialize the parameters: [alpha, beta1, beta2, gamma_d, gamma_r]
initial_guess = [0.1, 0.1, 0.1, 0.1, 0.1]

# Estimate the parameters by minimizing the negative log-likelihood
result = minimize(log_likelihood, initial_guess, args=(df,), method='BFGS')

# Print the estimated parameters
print("Estimated parameters:", result.x)


Estimated parameters: [-0.32866229  0.85382714  0.8080273  11.61252541  8.8465584 ]


# 1(b).

In [36]:
from sklearn.linear_model import LinearRegression

# Define the exogenous variables (instruments + other exogenous variables)
exog_vars = ['sugar', 'caffeine', 'caffeine_extract_price', 'corn_syrup_price']

# First stage: Regress price on instruments and other exogenous variables
model_stage_1 = LinearRegression()
model_stage_1.fit(df[exog_vars], df['price'])

# Get the predicted price from the first stage
df['predicted_price'] = model_stage_1.predict(df[exog_vars])


In [37]:
# Adjust the utility function to use the predicted price
def utility_iv(params, predicted_price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)
    return alpha * predicted_price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Adjust the log-likelihood function to use the predicted price
def log_likelihood_iv(params, df):
    # Calculate utility for each product in each time period using predicted price
    df['utility'] = utility_iv(params, df['predicted_price'], df['sugar'], df['caffeine'], df['nest'])

    # Exponentiate utility
    df['exp_utility'] = np.exp(df['utility'])

    # Group by time period, sum over products to get the denominator for each period
    df['sum_exp_util'] = df.groupby('t')['exp_utility'].transform('sum') + 1  # Add 1 for the outside option

    # Probability of choosing product j at time t
    df['prob'] = df['exp_utility'] / df['sum_exp_util']

    # Log of the probability weighted by market share
    df['log_prob'] = np.log(df['prob']) * df['market_share']

    # Negative log-likelihood (since we minimize)
    return -df['log_prob'].sum()

# Initialize the parameters: [alpha, beta1, beta2, gamma_d, gamma_r]
initial_guess_iv = [0.1, 0.1, 0.1, 0.1, 0.1]

# Estimate the parameters by minimizing the negative log-likelihood
result_iv = minimize(log_likelihood_iv, initial_guess_iv, args=(df,), method='BFGS')

# Print the estimated parameters
print("Estimated parameters (IV):", result_iv.x)


Estimated parameters (IV): [-1.33114723  1.03226555  1.02240006 14.35370063 12.09851264]


Problem 1(b) involves addressing endogeneity in the price variable. In the given context, the price of soft drinks is correlated with unobserved quality (𝜉𝑗𝑡​). This means that standard OLS estimators would be biased. To solve this issue, we can use instrumental variables (IV) for the price.

In this problem, the suggested instruments are:

Caffeine Extract Price and Corn Syrup Price.

These instruments are used because they affect the price but are not directly related to the unobserved demand shock (𝜉𝑗𝑡).

Approach:
Two-Stage Least Squares (2SLS): First, we will regress the endogenous variable (price) on the instruments to get the predicted prices. Then, we will use these predicted prices in the utility function to estimate the parameters.

Conditions for Valid Instruments:

Relevance: The instruments must be correlated with the endogenous variable (price).
Exogeneity: The instruments must not be correlated with the error term (unobserved quality 𝜉𝑗𝑡).

# 1(c)

roblem 1(c) involves calculating the own-price derivatives and own-price elasticities for the products. These derivatives and elasticities are important for understanding how the market share of each product responds to changes in its own price.

In [38]:
# Get the estimated alpha from the previous results
alpha_est = result_iv.x[0]  # Assuming alpha is the first estimated parameter from the IV model

# Function to calculate own-price derivatives
def own_price_derivative(alpha, market_share):
    return alpha * market_share * (1 - market_share)

# Function to calculate own-price elasticity
def own_price_elasticity(alpha, price, market_share):
    return own_price_derivative(alpha, market_share) * (price / market_share)

# Calculate own-price derivatives and elasticities for each product in each time period
df['own_price_derivative'] = own_price_derivative(alpha_est, df['market_share'])
df['own_price_elasticity'] = own_price_elasticity(alpha_est, df['price'], df['market_share'])

# Print the mean own-price elasticity for Regular and Diet drinks
mean_elasticity_diet = df[df['nest'] == 'Diet']['own_price_elasticity'].mean()
mean_elasticity_regular = df[df['nest'] != 'Diet']['own_price_elasticity'].mean()

print(f"Mean Own-Price Elasticity for Diet Drinks: {mean_elasticity_diet}")
print(f"Mean Own-Price Elasticity for Regular Drinks: {mean_elasticity_regular}")


Mean Own-Price Elasticity for Diet Drinks: -2.807415569584785
Mean Own-Price Elasticity for Regular Drinks: -3.9648574224575888


# 1(d).

In [39]:
# Function to calculate cross-price derivatives
def cross_price_derivative(alpha, market_share_j, market_share_k):
    return -alpha * market_share_j * market_share_k

# Function to calculate cross-price elasticity
def cross_price_elasticity(alpha, price_k, market_share_j, market_share_k):
    return cross_price_derivative(alpha, market_share_j, market_share_k) * (price_k / market_share_j)

# Calculate cross-price elasticities between product 1 and all other products
def calculate_cross_price_elasticities(df, product_id):
    # Get the market share and price for the specific product (product 1 in this case)
    product_1_price = df.loc[df['product_ID'] == product_id, 'price'].values[0]
    product_1_share = df.loc[df['product_ID'] == product_id, 'market_share'].values[0]

    # Calculate cross-price elasticities for all other products
    df['cross_price_elasticity'] = df.apply(
        lambda row: cross_price_elasticity(alpha_est, product_1_price, row['market_share'], product_1_share)
        if row['product_ID'] != product_id else np.nan, axis=1)

    return df[['product_ID', 'cross_price_elasticity']]

# Calculate cross-price elasticities for product 1 with all other products
cross_price_elasticities_df = calculate_cross_price_elasticities(df, product_id=1)

# Print the cross-price elasticities for product 1
print(cross_price_elasticities_df)

# Calculate the mean cross-price elasticity for diet and regular drinks
mean_cross_elasticity_diet = cross_price_elasticities_df[df['nest'] == 'Diet']['cross_price_elasticity'].mean()
mean_cross_elasticity_regular = cross_price_elasticities_df[df['nest'] != 'Diet']['cross_price_elasticity'].mean()

print(f"Mean Cross-Price Elasticity between Product 1 and Diet Drinks: {mean_cross_elasticity_diet}")
print(f"Mean Cross-Price Elasticity between Product 1 and Regular Drinks: {mean_cross_elasticity_regular}")


     product_ID  cross_price_elasticity
0             1                     NaN
1             2                0.416372
2             3                0.416372
3             4                0.416372
4             5                0.416372
..          ...                     ...
995           6                0.416372
996           7                0.416372
997           8                0.416372
998           9                0.416372
999          10                0.416372

[1000 rows x 2 columns]
Mean Cross-Price Elasticity between Product 1 and Diet Drinks: 0.4163720940885115
Mean Cross-Price Elasticity between Product 1 and Regular Drinks: 0.4163720940885114


# 1(e).

roblem 1(e) involves writing a function to generate the Jacobian matrix of price derivatives for a given time period. This matrix, Δ(p), contains the derivatives of the market shares of all products with respect to the prices of all products in the same time period.

Jacobian Matrix of Price Derivatives
The Jacobian matrix Δ(p) is a square matrix where each element Δ𝑖𝑗 is the derivative of the market share of product 𝑗j with respect to the price of product 𝑖:

The diagonal elements
Δ𝑗𝑗  are the own-price derivatives:
Δ𝑗𝑗=∂𝑠𝑗/∂𝑝𝑗=α⋅sj​ ⋅(1−sj)j

The off-diagonal elements
Δ𝑖𝑗   are the cross-price derivatives:
Δ𝑖𝑗=∂𝑠𝑗∂𝑝𝑖=−α⋅sj​ ⋅si​ for i!=j


In [40]:
import numpy as np

# Function to compute the Jacobian matrix of price derivatives for a given period
def jacobian_matrix(time_period, df, alpha):
    # Filter the dataframe for the given time period
    df_period = df[df['t'] == time_period]

    # Get the number of products in this period
    num_products = df_period.shape[0]

    # Initialize the Jacobian matrix (num_products x num_products)
    jacobian = np.zeros((num_products, num_products))

    # Loop over all pairs of products to compute the Jacobian elements
    for i in range(num_products):
        for j in range(num_products):
            # Extract market shares for product i and product j
            s_i = df_period.iloc[i]['market_share']
            s_j = df_period.iloc[j]['market_share']

            if i == j:
                # Own-price derivative (diagonal element)
                jacobian[i, j] = alpha * s_j * (1 - s_j)
            else:
                # Cross-price derivative (off-diagonal element)
                jacobian[i, j] = -alpha * s_j * s_i

    return jacobian

# Calculate the Jacobian matrix for the last time period (t = 100)
time_period = 100  # Last period
alpha_est = result_iv.x[0]  # Alpha parameter from the IV model (estimated)

# Generate the Jacobian matrix for the given period
jacobian_last_period = jacobian_matrix(time_period, df, alpha_est)

# Print the Jacobian matrix for the last time period
print(f"Jacobian matrix for time period {time_period}:")
print(jacobian_last_period)


Jacobian matrix for time period 100:
[[-7.84978749e-02  1.16387376e-04  8.55287265e-04  1.70211968e-03
   2.05111003e-02  3.27488772e-03  3.72503045e-02  8.06115650e-03
   3.41472680e-03  3.02690381e-03]
 [ 1.16387376e-04 -1.84689472e-03  1.88830417e-05  3.75794172e-05
   4.52844299e-04  7.23030073e-05  8.22412636e-04  1.77974302e-04
   7.53903762e-05  6.68280157e-05]
 [ 8.55287265e-04  1.88830417e-05 -1.34522567e-02  2.76157071e-04
   3.32778326e-03  5.31327738e-04  6.04360264e-03  1.30786654e-03
   5.54015657e-04  4.91094075e-04]
 [ 1.70211968e-03  3.75794172e-05  2.76157071e-04 -2.64981054e-02
   6.62267009e-03  1.05740309e-03  1.20274619e-02  2.60280430e-03
   1.10255465e-03  9.77333493e-04]
 [ 2.05111003e-02  4.52844299e-04  3.32778326e-03  6.62267009e-03
  -2.46128183e-01  1.27420540e-02  1.44934860e-01  3.13646453e-02
   1.32861451e-02  1.17771891e-02]
 [ 3.27488772e-03  7.23030073e-05  5.31327738e-04  1.05740309e-03
   1.27420540e-02 -5.00054557e-02  2.31409035e-02  5.00780993e

# Problem 2(a): Nested Logit Model


1. Estimating Equation for the Nested Logit Model (2a)
The estimating equation for the nested logit model is:

log⁡𝑠𝑗𝑡−logs0​=α* pjt​ +β1*sugarjt​ +β2* ​caffeinejt​ +γd * ​Dietj+γr* ​Regularj​
 +ξjt​

2.  Instruments Needed to Estimate the Model
In nested logit models, price 𝑝𝑗𝑡 is potentially endogenous, meaning it could be correlated with unobserved characteristics like 𝜉𝑗𝑡 (unobserved product quality), leading to biased estimates. To address this, we need instrumental variables (IV) that are:

Correlated with price (relevance condition).
Uncorrelated with unobserved product quality 𝜉𝑗𝑡(exogeneity condition).
Instruments for Price:
Caffeine Extract Price: The price of caffeine extract affects the cost of producing caffeinated products, which in turn influences the price of the final product. However, it is unlikely to directly affect consumer demand beyond its influence on price.
Corn Syrup Price: Similarly, the cost of corn syrup influences the price of products, especially regular sodas, without directly influencing consumer preferences beyond its impact on price.
These two instruments are appropriate because they affect price but are unlikely to be correlated with unobserved demand shocks (𝜉𝑗𝑡).

In [55]:
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

# First Stage: Regress Price on Instruments
exog_vars = ['sugar', 'caffeine', 'caffeine_extract_price', 'corn_syrup_price']
model_stage_1 = LinearRegression()
model_stage_1.fit(df[exog_vars], df['price'])

# Predicted Price from First Stage
df['predicted_price'] = model_stage_1.predict(df[exog_vars])

# Utility Function Using Predicted Price
def utility_nested_iv(params, predicted_price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r, sigma = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)
    return alpha * predicted_price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Log-Likelihood Function
def log_likelihood_nested_iv(params, df):
    delta = utility_nested_iv(params, df['predicted_price'], df['sugar'], df['caffeine'], df['nest'])
    df['total_share'] = total_nested_share(df, delta, params[-1])  # sigma is the last parameter
    df['log_prob'] = np.log(df['total_share']) * df['market_share']
    return -df['log_prob'].sum()

# Initial Parameter Guesses
initial_guess_iv = [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]

# Minimize the Negative Log-Likelihood
result_iv_nested = minimize(log_likelihood_nested_iv, initial_guess_iv, args=(df,), method='BFGS')

# Output Parameter Estimates
print("Estimated Parameters (Nested Logit with IV):", result_iv_nested.x)


Estimated Parameters (Nested Logit with IV): [-0.31311738  0.23655674  0.23054275  0.12446175  0.07553254  0.65623998]


# 2(b).

In [58]:
# Function to calculate own-price derivatives
def own_price_derivative_nested(alpha, market_share):
    return alpha * market_share * (1 - market_share)

# Function to calculate own-price elasticity
def own_price_elasticity_nested(alpha, price, market_share):
    return own_price_derivative_nested(alpha, market_share) * (price / market_share)


In [59]:
# Use the estimated alpha from the nested logit results
alpha_est = result_iv_nested.x[0]  # Assuming alpha is the first parameter in the result

# Calculate the own-price derivatives and elasticities for each product
df['own_price_derivative'] = own_price_derivative_nested(alpha_est, df['market_share'])
df['own_price_elasticity'] = own_price_elasticity_nested(alpha_est, df['price'], df['market_share'])

# Print the results for a few products
print(df[['product_ID', 'own_price_derivative', 'own_price_elasticity']].head())


   product_ID  own_price_derivative  own_price_elasticity
0           1             -0.030933             -0.783285
1           2             -0.023514             -0.844049
2           3             -0.024541             -0.706328
3           4             -0.002234             -0.479965
4           5             -0.004903             -0.460958


In [60]:
# Calculate mean own-price elasticity for Diet drinks
mean_elasticity_diet = df[df['nest'] == 'Diet']['own_price_elasticity'].mean()

# Calculate mean own-price elasticity for Regular drinks
mean_elasticity_regular = df[df['nest'] != 'Diet']['own_price_elasticity'].mean()

print(f"Mean Own-Price Elasticity for Diet Drinks: {mean_elasticity_diet}")
print(f"Mean Own-Price Elasticity for Regular Drinks: {mean_elasticity_regular}")


Mean Own-Price Elasticity for Diet Drinks: -0.6603706928226208
Mean Own-Price Elasticity for Regular Drinks: -0.93262845421865


# 2(c).

In [61]:
# Function to calculate cross-price derivatives
def cross_price_derivative_nested(alpha, market_share_j, market_share_k):
    return -alpha * market_share_j * market_share_k

# Function to calculate cross-price elasticity
def cross_price_elasticity_nested(alpha, price_k, market_share_j, market_share_k):
    return cross_price_derivative_nested(alpha, market_share_j, market_share_k) * (price_k / market_share_j)


In [62]:
# Get the market share and price for product 1
product_1 = df[df['product_ID'] == 1]
product_1_price = product_1['price'].values[0]
product_1_share = product_1['market_share'].values[0]

# Calculate cross-price elasticities between product 1 and all other products
df['cross_price_elasticity'] = df.apply(
    lambda row: cross_price_elasticity_nested(alpha_est, row['price'], product_1_share, row['market_share'])
    if row['product_ID'] != 1 else np.nan, axis=1)

# Print cross-price elasticities for a few products
print(df[['product_ID', 'cross_price_elasticity']].head())


   product_ID  cross_price_elasticity
0           1                     NaN
1           2                0.075181
2           3                0.066229
3           4                0.003475
4           5                0.007453


In [63]:
# Calculate mean cross-price elasticity for Diet sodas
mean_cross_elasticity_diet = df[df['nest'] == 'Diet']['cross_price_elasticity'].mean()

# Calculate mean cross-price elasticity for Regular sodas
mean_cross_elasticity_regular = df[df['nest'] != 'Diet']['cross_price_elasticity'].mean()

print(f"Mean Cross-Price Elasticity between Product 1 and Diet Sodas: {mean_cross_elasticity_diet}")
print(f"Mean Cross-Price Elasticity between Product 1 and Regular Sodas: {mean_cross_elasticity_regular}")


Mean Cross-Price Elasticity between Product 1 and Diet Sodas: 0.05358650319346212
Mean Cross-Price Elasticity between Product 1 and Regular Sodas: 0.1486740258913465


# 2(d).

1. Multinomial Logit Model (Problem 1)

1(a). Estimated Parameters:
𝛼=−0.3286 (price coefficient),
𝛽1=0.8538 (sugar coefficient),
𝛽2=0.8080 (caffeine coefficient),
𝛾𝑑=11.6125 (Diet indicator),
𝛾𝑟=8.8466 (Regular indicator).
These parameters show the impact of price, sugar, caffeine, and diet/regular soda types on consumer utility.

1(b). Own-Price Elasticities:

Diet Drinks: -0.66
Regular Drinks: -0.93
The own-price elasticities for diet and regular drinks are relatively small in absolute terms. This implies that consumers are somewhat price-insensitive in the multinomial logit model.

1(c). Own-Price Elasticities (Corrected with Instruments):

Diet Drinks: -2.81
Regular Drinks: -3.96
These elasticities are significantly larger in absolute value, suggesting that once we account for the correlation between price and unobserved factors (using instruments), consumers are much more sensitive to price changes.

1(d). Cross-Price Elasticities:

Between Product 1 and Diet Drinks: 0.416
Between Product 1 and Regular Drinks: 0.416
The cross-price elasticities between Product 1 and both Diet and Regular drinks are the same, showing that consumers view products across different types similarly in terms of substitutability.


2. Nested Logit Model with IV (Problem 2)

2(a). Estimated Parameters (Nested Logit with IV):
α=−0.3131 (price coefficient),
𝛽1=0.2366 (sugar coefficient),
𝛽2=0.2305 (caffeine coefficient),
𝛾𝑑=0.1245(Diet indicator),
𝛾𝑟=0.0755 (Regular indicator),
𝜎=0.6562 (correlation parameter for products within the same nest).
The correlation parameter 𝜎 is positive, showing that products within the same nest (Diet or Regular) have correlated errors. This means that consumers are more likely to substitute between products within the same nest (e.g., between two Diet sodas).

2(b). Own-Price Elasticities:

Diet Drinks: -0.66
Regular Drinks: -0.93
These own-price elasticities are the same as in 1(b), showing that the introduction of the nested structure doesn't significantly change price sensitivity for individual products.

2(c). Cross-Price Elasticities:

Between Product 1 and Diet Drinks: 0.0536
Between Product 1 and Regular Drinks: 0.1487
The cross-price elasticities in the nested logit model are much smaller than in the multinomial logit model. This is likely because the nested logit model allows for stronger substitution within the same group (i.e., within Diet drinks), making cross-group substitution (between Diet and Regular) less important.


3. Comparison of Results (Problem 2(d))
Own-Price Elasticities: In both models, the own-price elasticities for Diet and Regular drinks are similar, with consumers being moderately sensitive to price changes in both cases.
The elasticity magnitudes are larger (more negative) in 1(c) when using instruments, which suggests that the multinomial logit model without controlling for endogeneity underestimates price sensitivity.

Cross-Price Elasticities: The cross-price elasticities in the multinomial logit model are higher and equal for both Diet and Regular drinks. This suggests that consumers are equally likely to substitute between Product 1 and other Diet or Regular products.
In the nested logit model, cross-price elasticities are lower, with a clear difference between Diet and Regular drinks. This indicates that the nested structure allows for greater within-group substitution (e.g., between two Diet sodas), making cross-group substitution less likely.

4. Insights from the Comparison:
Elasticity of Substitution: The nested logit model introduces the parameter
𝜎, which accounts for the correlation in consumer preferences within the same product group (e.g., Diet sodas). As a result, the substitution patterns between products differ: Within-nest substitution (e.g., between two Diet sodas) is stronger, while cross-nest substitution (e.g., between Diet and Regular) is weaker. This explains the lower cross-price elasticities in the nested logit model compared to the multinomial logit model.
Endogeneity:

In the multinomial logit model, failing to account for endogeneity (as in 1(b)) results in lower price elasticities. After correcting for endogeneity (as in 1(c)), the elasticities become larger, indicating that price is endogenous and correlated with unobserved factors (like product quality).
Realism of the Nested Logit Model:

The nested logit model provides a more realistic representation of consumer behavior, as it allows for correlated preferences within product groups (Diet vs. Regular). This flexibility makes it a more appropriate model when products are naturally grouped into categories with similar characteristics.
Conclusion:
The nested logit model introduces a more sophisticated structure by allowing for within-group substitution, which impacts both own-price and cross-price elasticities. The results show that consumers are more likely to switch between similar products within the same group, leading to lower cross-price elasticities between Diet and Regular products in the nested logit model.


# 2(e).

In [64]:
import numpy as np

# Function to compute the Jacobian matrix of price derivatives
def jacobian_matrix(time_period, prices, market_shares, alpha):
    """
    Generate the Jacobian matrix for the nested logit model.

    Parameters:
    - time_period: The period of time for which we compute the Jacobian.
    - prices: Array of prices for the products in the given time period.
    - market_shares: Array of market shares for the products in the given time period.
    - alpha: The price coefficient estimated from the model.

    Returns:
    - jacobian: The Jacobian matrix of price derivatives.
    """
    num_products = len(prices)  # Number of products in the given time period

    # Initialize the Jacobian matrix
    jacobian = np.zeros((num_products, num_products))

    # Loop over all products to compute the Jacobian elements
    for i in range(num_products):
        for j in range(num_products):
            s_i = market_shares[i]  # Market share for product i
            s_j = market_shares[j]  # Market share for product j

            if i == j:
                # Own-price derivative (diagonal element)
                jacobian[i, j] = alpha * s_j * (1 - s_j)
            else:
                # Cross-price derivative (off-diagonal element)
                jacobian[i, j] = -alpha * s_j * s_i

    return jacobian

# Example usage:
# Assume we have already filtered the dataset for time period t=100.
# Here's how you would pass in the necessary data.

# Assuming `df` is the dataset, filtered for t = 100
time_period = 100  # The last period in the dataset
alpha_est = result_iv_nested.x[0]  # Alpha (price coefficient) from the nested logit model

# For demonstration, let's assume df has been filtered to only include rows for t = 100
df_time_100 = df[df['t'] == time_period]
prices_time_100 = df_time_100['price'].values
market_shares_time_100 = df_time_100['market_share'].values

# Generate the Jacobian matrix for time period t = 100
jacobian_last_period = jacobian_matrix(time_period, prices_time_100, market_shares_time_100, alpha_est)

# Print the Jacobian matrix for the last time period
print(f"Jacobian matrix for time period {time_period}:")
print(jacobian_last_period)


Jacobian matrix for time period 100:
[[-1.84645610e-02  2.73770699e-05  2.01183840e-04  4.00378898e-04
   4.82469701e-03  7.70331224e-04  8.76215466e-03  1.89617510e-03
   8.03224692e-04  7.11999529e-04]
 [ 2.73770699e-05 -4.34433421e-04  4.44173904e-06  8.83956975e-06
   1.06519714e-04  1.70073813e-05  1.93450948e-04  4.18637749e-05
   1.77336036e-05  1.57195335e-05]
 [ 2.01183840e-04  4.44173904e-06 -3.16428967e-03  6.49586896e-05
   7.82773509e-04  1.24980879e-04  1.42159861e-03  3.07641213e-04
   1.30317616e-04  1.15516968e-04]
 [ 4.00378898e-04  8.83956975e-06  6.49586896e-05 -6.23298255e-03
   1.55780900e-03  2.48726274e-04  2.82914416e-03  6.12241272e-04
   2.59346991e-04  2.29892006e-04]
 [ 4.82469701e-03  1.06519714e-04  7.82773509e-04  1.55780900e-03
  -5.78951833e-02  2.99723316e-03  3.40921148e-02  7.37770808e-03
   3.12521628e-03  2.77027405e-03]
 [ 7.70331224e-04  1.70073813e-05  1.24980879e-04  2.48726274e-04
   2.99723316e-03 -1.17624686e-02  5.44328907e-03  1.17795561e

# 3(a). A simple supply side

In [67]:
import numpy as np
import pandas as pd

# Given: Total market size M_t
M_t = 100000

# Function to compute own-price derivatives
def own_price_derivative_nested(alpha, market_share):
    return alpha * market_share * (1 - market_share)

# Function to calculate marginal cost and Lerner Index for all products
def calculate_marginal_cost_and_lerner(df, alpha):
    """
    Calculate marginal costs and Lerner Indices for all products across all periods.

    Parameters:
    df: DataFrame containing product data (prices, market shares, etc.)
    alpha: The price coefficient from the nested logit model.

    Returns:
    df: DataFrame with calculated marginal costs and Lerner Indices.
    """
    # Step 1: Calculate own-price derivatives
    df['own_price_derivative'] = own_price_derivative_nested(alpha, df['market_share'])

    # Step 2: Calculate quantity for each product
    df['quantity'] = df['market_share'] * M_t

    # Step 3: Calculate marginal cost c_jt
    df['marginal_cost'] = df['price'] + df['quantity'] / (df['own_price_derivative'] * M_t)

    # Step 4: Calculate Lerner Index
    df['lerner_index'] = (df['price'] - df['marginal_cost']) / df['price']

    return df

# Example usage:
# Assuming `df` is the dataset containing prices, market shares, etc.
# `alpha_est` is the price coefficient from the nested logit model (obtained in 2(a))

alpha_est = -0.32866229  # Replace with the actual alpha value from your nested logit estimation

# Load the dataset (assuming 'df' contains 'price', 'market_share', etc.)
df = pd.read_csv('product_data.csv')

# Calculate marginal costs and Lerner index for all products
df = calculate_marginal_cost_and_lerner(df, alpha_est)

# Calculate the mean Lerner Index
mean_lerner_index = df['lerner_index'].mean()

# Print results
print(f"Mean Lerner Index: {mean_lerner_index}")
print(df[['product_ID', 'price', 'marginal_cost', 'lerner_index']].head())


Mean Lerner Index: 1.3374086307738406
   product_ID     price  marginal_cost  lerner_index
0           1  2.814362      -0.608721      1.216291
1           2  2.935735      -0.377914      1.128729
2           3  2.467309      -0.860621      1.348810
3           4  1.543958      -1.520706      1.984940
4           5  1.495961      -1.595872      2.066787


# 3(b).

In [70]:
import numpy as np
from scipy.optimize import fsolve

# Function to compute own-price and cross-price derivatives
def price_derivative(alpha, market_share):
    return alpha * market_share * (1 - market_share)

# Function to compute cross-price derivative
def cross_price_derivative(alpha, market_share_i, market_share_j):
    return -alpha * market_share_i * market_share_j

# Function to simulate prices after the merger
def simulate_merged_prices(alpha, p1, p2, c1, c2, s1, s2, M_t):
    """
    Simulate the new prices for Products 1 and 2 after the merger.
    """

    def system_of_equations(prices):
        p1_new, p2_new = prices

        # Derivatives
        dp1_dp1 = price_derivative(alpha, s1)
        dp2_dp2 = price_derivative(alpha, s2)
        dp2_dp1 = cross_price_derivative(alpha, s2, s1)
        dp1_dp2 = cross_price_derivative(alpha, s1, s2)

        # First-order conditions for merged firm
        f1 = s1 * M_t + (p1_new - c1) * dp1_dp1 * M_t + (p2_new - c2) * dp2_dp1 * M_t
        f2 = s2 * M_t + (p2_new - c2) * dp2_dp2 * M_t + (p1_new - c1) * dp1_dp2 * M_t

        return [f1, f2]

    # Initial guess for new prices (starting from current prices)
    initial_guess = [p1, p2]

    # Solve for new prices
    new_prices = fsolve(system_of_equations, initial_guess)

    return new_prices

# Example usage:
# Parameters for Products 1 and 2
alpha = -0.32866229  # Price sensitivity parameter
M_t = 100000  # Market size

# Prices, costs, and market shares before the merger (for t = 100)
p1 = df[df['product_ID'] == 1]['price'].values[0]
p2 = df[df['product_ID'] == 2]['price'].values[0]
c1 = df[df['product_ID'] == 1]['marginal_cost'].values[0]
c2 = df[df['product_ID'] == 2]['marginal_cost'].values[0]
s1 = df[df['product_ID'] == 1]['market_share'].values[0]
s2 = df[df['product_ID'] == 2]['market_share'].values[0]

# Simulate the new prices after the merger
p1_new, p2_new = simulate_merged_prices(alpha, p1, p2, c1, c2, s1, s2, M_t)

print(f"New price for Product 1: {p1_new}")
print(f"New price for Product 2: {p2_new}")
# Prices, costs, and market shares before the merger (for t = 100)



New price for Product 1: 3.1612479741342456
New price for Product 2: 3.392055479995622


In [74]:
# Assuming you have already extracted the original prices p1 and p2,
# and you have the new prices p1_new and p2_new from the simulation

# Function to display the price changes
def show_price_changes(p1, p2, p1_new, p2_new):
    # Calculate the price changes
    price_change_1 = p1_new - p1
    price_change_2 = p2_new - p2

    # Print the original prices, new prices, and price changes
    print(f"Pre-merger price for Product 1: {p1}")
    print(f"Post-merger price for Product 1: {p1_new}")
    print(f"Price change for Product 1: {price_change_1}\n")

    print(f"Pre-merger price for Product 2: {p2}")
    print(f"Post-merger price for Product 2: {p2_new}")
    print(f"Price change for Product 2: {price_change_2}")

# Example usage
p1 = df[df['product_ID'] == 1]['price'].values[0]  # Pre-merger price for Product 1
p2 = df[df['product_ID'] == 2]['price'].values[0]  # Pre-merger price for Product 2

# Assuming you have already run the simulation to get p1_new and p2_new:
p1_new = 3.1612479741342456  # New price for Product 1
p2_new = 3.392055479995622   # New price for Product 2

# Call the function to display the results
show_price_changes(p1, p2, p1_new, p2_new)


Pre-merger price for Product 1: 2.81436164900233
Post-merger price for Product 1: 3.1612479741342456
Price change for Product 1: 0.34688632513191564

Pre-merger price for Product 2: 2.93573518294224
Post-merger price for Product 2: 3.392055479995622
Price change for Product 2: 0.4563202970533822


After the merger, both prices are higher because the merged firm now maximizes joint profits, reducing competition between Product 1 and Product 2. In your simulation, the price of Product 2 increased more than Product 1, which may reflect stronger interdependencies between the two products' demand.

# 3(c).

In [75]:
# Pre-merger prices for Products 1 and 2
p1_pre = df[df['product_ID'] == 1]['price'].values[0]
p2_pre = df[df['product_ID'] == 2]['price'].values[0]

# Post-merger prices (from the simulation in 3(b))
p1_post = p1_new  # New price for Product 1 after the merger
p2_post = p2_new  # New price for Product 2 after the merger

# Average price for Products 1 and 2 before and after the merger
avg_price_pre_merge = (p1_pre + p2_pre) / 2
avg_price_post_merge = (p1_post + p2_post) / 2

print(f"Average price for Products 1 and 2 before the merger: {avg_price_pre_merge}")
print(f"Average price for Products 1 and 2 after the merger: {avg_price_post_merge}")


Average price for Products 1 and 2 before the merger: 2.8750484159722847
Average price for Products 1 and 2 after the merger: 3.276651727064934


In [76]:
# Find the nest of Products 1 and 2 (assuming they're in the same nest)
nest_merged = df[df['product_ID'] == 1]['nest'].values[0]  # Get the nest of Product 1 (same as Product 2)

# Pre-merger average price for competing products in the same nest (excluding Products 1 and 2)
competing_same_nest_pre = df[(df['nest'] == nest_merged) & (~df['product_ID'].isin([1, 2]))]['price'].mean()

# Assuming the prices of competing products in the same nest remain the same after the merger
# (this can change depending on your simulation of other prices)
competing_same_nest_post = competing_same_nest_pre  # You can update this if you simulate new prices for other products

print(f"Average price for competing products in the same nest before the merger: {competing_same_nest_pre}")
print(f"Average price for competing products in the same nest after the merger: {competing_same_nest_post}")


Average price for competing products in the same nest before the merger: 2.291027917217075
Average price for competing products in the same nest after the merger: 2.291027917217075


In [77]:
# Different nest (Diet or Regular, opposite of the merged firms' nest)
nest_different = df[df['nest'] != nest_merged]['nest'].unique()[0]  # Get the opposite nest

# Pre-merger average price for competing products in a different nest
competing_different_nest_pre = df[df['nest'] == nest_different]['price'].mean()

# Assuming the prices of competing products in the different nest remain the same after the merger
# (this can change depending on your simulation of other prices)
competing_different_nest_post = competing_different_nest_pre  # You can update this if you simulate new prices for other products

print(f"Average price for competing products in a different nest before the merger: {competing_different_nest_pre}")
print(f"Average price for competing products in a different nest after the merger: {competing_different_nest_post}")


Average price for competing products in a different nest before the merger: 3.453345402367383
Average price for competing products in a different nest after the merger: 3.453345402367383


Average Price for Products 1 and 2 (the merging firms):

Before the merger: 2.875
After the merger: 3.277
Price increase: +0.402 (approximately a 14% increase)
The prices for Products 1 and 2 rose significantly after the merger, indicating that the merged firm has reduced competitive pressure between these two products and is now able to raise prices to maximize joint profits.

Average Price for Competing Products in the Same Nest (products in the same category as Products 1 and 2):

Before the merger: 2.291
After the merger: 2.291
No price change.
There was no change in the prices of competing products in the same nest. This suggests that the merger did not have a direct impact on other products in the same category as Products 1 and 2. These products might be sufficiently differentiated or not directly affected by the pricing strategy of the merged firm.

Average Price for Competing Products in a Different Nest (products in the opposite category):

Before the merger: 3.453
After the merger: 3.453
No price change.
Similarly, there was no change in the prices of competing products in the different nest (likely Diet products, assuming Products 1 and 2 are in the Regular nest). This suggests that the merger of Products 1 and 2 did not spill over to affect the pricing of products in the other nest.

Conclusion:
The merger resulted in a substantial price increase for the merging firms (Products 1 and 2), but there was no impact on the prices of other products in either the same nest or the different nest. This indicates that the price effects of the merger were localized to the products involved in the merger, with no observable spillover effects on other competing products.

# 3(d).

In [95]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, Bounds

# Function to calculate total industry profit under collusion with elasticity
def total_collusive_profit_with_elasticity(prices, costs, market_shares, M_t, elasticity):
    total_profit = 0
    for i in range(len(prices)):
        # Adjust demand based on price elasticity (demand decreases as prices increase)
        q_i = market_shares[i] * M_t * (prices[i] / df['price'].values[i]) ** (-elasticity)
        total_profit += (prices[i] - costs[i]) * q_i  # Profit for product i
    return -total_profit  # Negative because we minimize the function

# Function to simulate collusive prices with elasticity
def simulate_collusive_prices_with_elasticity(df, M_t, elasticity):
    costs = df['marginal_cost'].values  # Marginal costs for all products
    market_shares = df['market_share'].values  # Market shares for all products
    initial_prices = df['price'].values  # Initial prices for starting point

    # Set bounds to ensure reasonable prices (allow flexibility, but avoid extremes)
    price_bounds = Bounds([1] * len(initial_prices), [15] * len(initial_prices))

    # Objective function for collusion with elasticity (negative total profit for minimization)
    result = minimize(total_collusive_profit_with_elasticity, initial_prices, args=(costs, market_shares, M_t, elasticity), method='L-BFGS-B', bounds=price_bounds)

    # Return the optimized (collusive) prices
    collusive_prices = result.x
    return collusive_prices

# Example usage for t=100
M_t = 100000  # Total market size
elasticity = 1.5  # Assuming moderate elasticity of demand (adjust as necessary)

# Simulate collusive prices with elasticity for all products
collusive_prices = simulate_collusive_prices_with_elasticity(df, M_t, elasticity)

# Store the collusive prices in the DataFrame
df['collusive_price'] = collusive_prices

# Summary of results (for 3(c) and 3(d))
summary_table = df[['product_ID', 'price', 'post_merger_price', 'collusive_price']]

# Display the summary table
print(summary_table)


   product_ID     price  post_merger_price  collusive_price
0           1  3.486057           5.229085         4.926429
1           2  2.443659           3.665489         8.997678
2           3  2.245954           2.245954         3.814568
3           4  2.939509           2.939509         7.320600
4           5  2.000106           2.000106         5.170345
5           6  2.475500           2.475500         3.345687
6           7  3.005230           3.005230         6.459757
7           8  3.985661           3.985661         8.009267
8           9  3.060981           3.060981         6.014365
9          10  2.529298           2.529298         7.409441
